In [60]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [61]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [19]:
Customer_Details_Query='''

SELECT *
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
--where last_updt_ts >='2022-08-01'  and last_updt_ts < '2022-10-01'
where date(last_updt_ts) ='2022-09-30'
and account_type_name!='Standalone'
and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id


'''

In [20]:
Customer_DF=extract_bq_data(bq_client, sql=Customer_Details_Query)

In [21]:
Customer_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296739 entries, 0 to 296738
Data columns (total 50 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   dealer_id                         296739 non-null  Int64              
 1   dealer_name                       296739 non-null  object             
 2   dealer_branch_id                  20833 non-null   object             
 3   dealer_branch_desc                296739 non-null  object             
 4   customer_id                       296739 non-null  Int64              
 5   title                             296739 non-null  object             
 6   lastname                          296739 non-null  object             
 7   firstname                         296739 non-null  object             
 8   address1                          296739 non-null  object             
 9   address2                          296739 non-nul

In [22]:
Customer_DF.head()

,dealer_id,dealer_name,dealer_branch_id,dealer_branch_desc,customer_id,title,lastname,firstname,address1,address2,...,install_type_id,install_type_desc,builder_program_model_home,primary_email,primary_phone,file_name,create_ts,create_user_id,last_updt_ts,last_updt_user_id
0,15525,TELUS Communications Inc.,None,,1012333,,Ferguson,Daniel,156 West 11th Ave,,...,1.000000000,New Professionally Installed System,False,AaxCA6aEU1GLz97qIyHK9jHtLJEoHUkRqO4WlNKt9Tog,+14039862174,CustomerAccountDetails-20220929,2022-09-30 11:27:24.729296+00:00,etl_cust,2022-09-30 11:27:24.729296+00:00,etl_cust
1,15525,TELUS Communications Inc.,None,,1123473,,Wong,Stephanie,4106 B Torquay Drive,,...,1.000000000,New Professionally Installed System,False,Af3Dm2vf3YWkewtnswcifJXDG1Qwuyjfa8p32Buv2nM+jDY=,+17788895882,CustomerAccountDetails-20220929,2022-09-30 11:27:24.729296+00:00,etl_cust,2022-09-30 11:27:24.729296+00:00,etl_cust
2,15525,TELUS Communications Inc.,None,,1189877,,McKenzie,Sylvia,45 Roland Street,,...,1.000000000,New Professionally Installed System,<NA>,Abn4b7rNvCV5u1wdxaTtVb2+8cr97PHp5BzIcbp+rnYUxw==,+14033427729,CustomerAccountDetails-20220929,2022-09-30 11:27:24.729296+00:00,etl_cust,2022-09-30 11:27:24.729296+00:00,etl_cust
3,15525,TELUS Communications Inc.,None,,1538476,,Dias,Vic,151 Andrew Close,,...,1.000000000,New Professionally Installed System,<NA>,AeJef6d/a1wo0K/le0HJP8raN2TD7ZR5FOJr45KrfhwSXV...,+14033589990,CustomerAccountDetails-20220929,2022-09-30 11:27:24.729296+00:00,etl_cust,2022-09-30 11:27:24.729296+00:00,etl_cust
4,15525,TELUS Communications Inc.,None,,1632664,,Hollman,Ed & Shelly,151 Andrew Close,,...,1.000000000,New Professionally Installed System,<NA>,AWXURtkv5shgODRLnbyxdyVpZTtH0ucOyhlllLyuA++N,+14033427706,CustomerAccountDetails-20220929,2022-09-30 11:27:24.729296+00:00,etl_cust,2022-09-30 11:27:24.729296+00:00,etl_cust


In [23]:
Customer_DF['dealer_name'].value_counts()

TELUS Communications Inc.    296739
Name: dealer_name, dtype: int64

In [38]:
Telus_Customer= '''

select bacct_bus_bacct_num,bacct_bacct_typ_cd,bacct_billg_acct_id,cust_bus_cust_id,pi_prev_bus_bacct_num,pi_prev_bus_bacct_num_src_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,pi_prod_instnc_stat_actvy_cd,pi_curr_bacct_assn_ts
-- ,param
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
-- CROSS JOIN UNNEST(list_tpl_pi_alias_str) as param
WHERE DATE(prod_instnc_ts) = "2022-09-30" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
-- and DATE(pi_prod_instnc_stat_ts)>='2022-08-01' and DATE(pi_prod_instnc_stat_ts)<'2022-09-01' #Date on which the status was updated
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id


'''

In [39]:
Telus_Customer_DF=extract_bq_data(bq_client, sql=Telus_Customer)

In [40]:
Telus_Customer_DF['Telus_flag']=1

In [41]:
Telus_Customer_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384083 entries, 0 to 384082
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   bacct_bus_bacct_num           384083 non-null  Int64         
 1   bacct_bacct_typ_cd            384083 non-null  object        
 2   bacct_billg_acct_id           384083 non-null  Int64         
 3   cust_bus_cust_id              384083 non-null  object        
 4   pi_prev_bus_bacct_num         384083 non-null  Int64         
 5   pi_prev_bus_bacct_num_src_id  384083 non-null  Int64         
 6   pi_prod_instnc_typ_cd         384083 non-null  object        
 7   pi_prod_instnc_stat_ts        384083 non-null  datetime64[ns]
 8   pi_prod_instnc_stat_actvy_cd  0 non-null       object        
 9   pi_curr_bacct_assn_ts         0 non-null       datetime64[ns]
 10  Telus_flag                    384083 non-null  int64         
dtypes: Int64(4), 

In [42]:
Merge_DF_1=Customer_DF.merge(Telus_Customer_DF,left_on='dealer_customer_id',right_on='cust_bus_cust_id',how='left')

In [43]:
Merge_DF_1['Telus_flag'].isna().sum()*100/Merge_DF_1.shape[0]

8.81279508254729

In [44]:
Merge_DF_1.head()

,dealer_id,dealer_name,dealer_branch_id,dealer_branch_desc,customer_id,title,lastname,firstname,address1,address2,...,bacct_bacct_typ_cd,bacct_billg_acct_id,cust_bus_cust_id,pi_prev_bus_bacct_num,pi_prev_bus_bacct_num_src_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,pi_prod_instnc_stat_actvy_cd,pi_curr_bacct_assn_ts,Telus_flag
0,15525,TELUS Communications Inc.,None,,1012333,,Ferguson,Daniel,156 West 11th Ave,,...,N/APP,99182621,99424688,-1000000000,-1000000000,SMHM,2019-06-20,None,NaT,1.0
1,15525,TELUS Communications Inc.,None,,1123473,,Wong,Stephanie,4106 B Torquay Drive,,...,N/APP,99182642,99424794,-1000000000,-1000000000,SMHM,2019-06-17,None,NaT,1.0
2,15525,TELUS Communications Inc.,None,,1189877,,McKenzie,Sylvia,45 Roland Street,,...,N/APP,101528695,100721513,-1000000000,-1000000000,SMHM,2020-05-22,None,NaT,1.0
3,15525,TELUS Communications Inc.,None,,1538476,,Dias,Vic,151 Andrew Close,,...,N/APP,101524595,100717708,-1000000000,-1000000000,SMHM,2020-05-21,None,NaT,1.0
4,15525,TELUS Communications Inc.,None,,1632664,,Hollman,Ed & Shelly,151 Andrew Close,,...,N/APP,101531666,100723625,-1000000000,-1000000000,SMHM,2020-05-22,None,NaT,1.0


In [45]:
Merge_DF_1['Telus_flag'].value_counts()

1.0    270588
Name: Telus_flag, dtype: int64

In [46]:
Merge_DF_1['pi_prod_instnc_typ_cd'].value_counts()

SMHM    270588
Name: pi_prod_instnc_typ_cd, dtype: int64

In [47]:
Merge_DF_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296739 entries, 0 to 296738
Data columns (total 61 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   dealer_id                         296739 non-null  Int64              
 1   dealer_name                       296739 non-null  object             
 2   dealer_branch_id                  20833 non-null   object             
 3   dealer_branch_desc                296739 non-null  object             
 4   customer_id                       296739 non-null  Int64              
 5   title                             296739 non-null  object             
 6   lastname                          296739 non-null  object             
 7   firstname                         296739 non-null  object             
 8   address1                          296739 non-null  object             
 9   address2                          296739 non-nul

In [56]:
cols_needed=['customer_id','dealer_customer_id','dealer_name','join_date','bacct_bus_bacct_num','bacct_bacct_typ_cd','bacct_billg_acct_id','cust_bus_cust_id','pi_prod_instnc_typ_cd','pi_prod_instnc_stat_ts','pi_prod_instnc_stat_actvy_cd','pi_curr_bacct_assn_ts']
Merge_DF_2=Merge_DF_1[cols_needed]

In [57]:
Merge_DF_2=Merge_DF_2[Merge_DF_1['Telus_flag']==1]

In [58]:
Merge_DF_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270588 entries, 0 to 296738
Data columns (total 12 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   customer_id                   270588 non-null  Int64              
 1   dealer_customer_id            270588 non-null  object             
 2   dealer_name                   270588 non-null  object             
 3   join_date                     270588 non-null  datetime64[ns, UTC]
 4   bacct_bus_bacct_num           270588 non-null  Int64              
 5   bacct_bacct_typ_cd            270588 non-null  object             
 6   bacct_billg_acct_id           270588 non-null  Int64              
 7   cust_bus_cust_id              270588 non-null  object             
 8   pi_prod_instnc_typ_cd         270588 non-null  object             
 9   pi_prod_instnc_stat_ts        270588 non-null  datetime64[ns]     
 10  pi_prod_instnc_stat_

In [59]:
Merge_DF_2.head()

,customer_id,dealer_customer_id,dealer_name,join_date,bacct_bus_bacct_num,bacct_bacct_typ_cd,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,pi_prod_instnc_stat_actvy_cd,pi_curr_bacct_assn_ts
0,1012333,99424688,TELUS Communications Inc.,2011-08-11 15:05:29+00:00,603917798,N/APP,99182621,99424688,SMHM,2019-06-20,None,NaT
1,1123473,99424794,TELUS Communications Inc.,2011-11-17 17:54:01+00:00,603917606,N/APP,99182642,99424794,SMHM,2019-06-17,None,NaT
2,1189877,100721513,TELUS Communications Inc.,2012-01-30 10:20:57+00:00,604262644,N/APP,101528695,100721513,SMHM,2020-05-22,None,NaT
3,1538476,100717708,TELUS Communications Inc.,2012-07-23 09:45:43+00:00,604260308,N/APP,101524595,100717708,SMHM,2020-05-21,None,NaT
4,1632664,100723625,TELUS Communications Inc.,2012-08-29 10:32:51+00:00,604263132,N/APP,101531666,100723625,SMHM,2020-05-22,None,NaT


In [55]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'ADC.ADC_Telus_Customers'

bq_table_instance= bq_client.load_table_from_dataframe(Merge_DF_2, Table_BQ,job_config=config)

/opt/conda/lib/python3.8/site-packages/google/cloud/bigquery/_pandas_helpers.py:558: UserWarning: Pyarrow could not determine the type of columns: pi_prod_instnc_stat_actvy_cd.
  warnings.warn(


In [ ]:
Merge_DF_1_non_match=Merge_DF_1[Merge_DF_1['Telus_flag'].isna()==True]

In [ ]:
Merge_DF_1_non_match.info()

In [ ]:
Merge_DF_1_non_match.head()

In [ ]:
Customer_Details_Adil_SQL='''

SELECT *
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) ='2022-09-30'
and account_type_name!='Standalone'
--and customer_type_name='Customer'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
order by dealer_name,join_date,customer_id


'''

In [ ]:
Customer_Details_Adil_DF=extract_bq_data(bq_client, sql=Customer_Details_Adil_SQL)

In [ ]:
Customer_Details_Adil_DF.info()

In [ ]:
Customer_Details_Adil_DF['customer_type_name'].value_counts()

In [ ]:
Customer_Details_Adil_DF['account_type_name'].value_counts()

In [ ]:
Customer_Details_Adil_DF['dealer_name'].value_counts()